<a href="https://colab.research.google.com/github/appliedcode/mthree-c422/blob/mthree-422-salleh/Exercises/day-14/Audit_GovernanceAI_Audit_Governance_Practice_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Problem Statement**

You are tasked with evaluating and auditing a machine learning–based **credit scoring model** to ensure it complies with **AI governance principles**, including fairness, transparency, and accountability.

The goal is to:

- Detect and measure potential **bias** in model predictions across demographic groups (in this case, `age_group` — younger vs older applicants).
- Provide **model transparency** through explainability tools such as **SHAP** to understand feature influences on decisions.
- Implement **AI auditing steps** by logging performance metrics, bias analysis, and interpretability findings, with proper documentation that could be used for **regulatory compliance** and internal governance processes.

You must train, evaluate, audit, and document the AI model in a way that aligns with **ethical AI adoption** in financial services.

**In this exercise, we will use the publicly available "Credit-G" dataset from OpenML.**

### **Dataset Collection Code**



In [5]:
# Install necessary packages
!pip install shap scikit-learn pandas -q

import numpy as np
import pandas as pd
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import shap
import datetime

In [6]:
from sklearn.datasets import fetch_openml

# Load dataset from OpenML
credit_data = fetch_openml(name='credit-g', version=1, as_frame=True)

# Features (X) and Target (y)
X = credit_data.data
y = credit_data.target

# Convert target labels to binary (1 = good credit, 0 = bad credit)
y = y.map({'good': 1, 'bad': 0})

print("Dataset shape:", X.shape)
print("Target distribution:\n", y.value_counts())

Dataset shape: (1000, 20)
Target distribution:
 class
1    700
0    300
Name: count, dtype: int64


In [12]:
# One-hot encode categorical columns
X_encoded = pd.get_dummies(X)
# N.B. This was added to get rid of error: "ValueError: could not convert string to float: 'no checking'" on "clf.fit(X_train, y_train)" below

# -------------------
# Train-test split
# -------------------
X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y, test_size=0.3, random_state=42
)

In [13]:
# -------------------
# Model Training
# -------------------
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

# Predictions
y_pred = clf.predict(X_test)

In [14]:
# -------------------
# Model Evaluation
# -------------------
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Classification Report:
               precision    recall  f1-score   support

           0       0.72      0.36      0.48        91
           1       0.77      0.94      0.85       209

    accuracy                           0.76       300
   macro avg       0.74      0.65      0.66       300
weighted avg       0.76      0.76      0.74       300



In [15]:
# -------------------
# AI Auditing Step 1: Log metrics
# -------------------
audit_log = {
    "timestamp": datetime.datetime.now().isoformat(),
    "classification_report": classification_report(y_test, y_pred, output_dict=True),
    "confusion_matrix": confusion_matrix(y_test, y_pred).tolist()
}
print("\nAudit Log Initial Entry:\n", audit_log)


Audit Log Initial Entry:
 {'timestamp': '2025-08-14T08:45:47.006558', 'classification_report': {'0': {'precision': 0.717391304347826, 'recall': 0.3626373626373626, 'f1-score': 0.48175182481751827, 'support': 91.0}, '1': {'precision': 0.7716535433070866, 'recall': 0.937799043062201, 'f1-score': 0.8466522678185745, 'support': 209.0}, 'accuracy': 0.7633333333333333, 'macro avg': {'precision': 0.7445224238274564, 'recall': 0.6502182028497818, 'f1-score': 0.6642020463180464, 'support': 300.0}, 'weighted avg': {'precision': 0.7551939974894443, 'recall': 0.7633333333333333, 'f1-score': 0.7359658001082541, 'support': 300.0}}, 'confusion_matrix': [[33, 58], [13, 196]]}


In [17]:
# -------------------
# Bias Detection by Age Group
# -------------------
test_results = X_test.copy()
test_results['actual'] = y_test
test_results['predicted'] = y_pred

grouped = test_results.groupby('age_group').agg(
    total=('actual', 'count'),
    positive_rate_actual=('actual', 'mean'),
    positive_rate_predicted=('predicted', 'mean')
)
print("\nOutcome rates by Age Group:\n", grouped)



KeyError: 'age_group'

In [ ]:
# Add bias report to audit log
bias_report = grouped.to_string()
audit_log['bias_report'] = bias_report

# -------------------
# Model Interpretability with SHAP
# -------------------
explainer = shap.TreeExplainer(clf)
shap_values = explainer.shap_values(X_test)

# Plot summary for positive class (good credit)
shap.summary_plot(shap_values[1], X_test)

# -------------------
# Compliance & Audit Documentation
# -------------------
compliance_notes = f"""
Model Compliance and Audit Report:
-----------------------------------
Timestamp: {audit_log['timestamp']}

Bias Detection:
Outcome disparities between age groups:
{bias_report}

Model Performance:
- Detailed metrics in classification report
- Confusion matrix: {audit_log["confusion_matrix"]}

Transparency:
- SHAP used to interpret feature importance and influence
- Explainability provided for decision accountability

Governance & Auditing:
- Metrics, bias analysis, and interpretability documented
- Ready for review under AI governance and financial regulatory frameworks
"""

print(compliance_notes)